In this experiment, you will explore the accuracy of sentiment classificaiton using different feature representations of text documents.

First, you will implement `createBasicFeatures`, which creates a sparse matrix representation of a collection of documents. For this exercise, you should have a feature for each word containing at least one alphabetic character. You may use the `numpy` and `sklearn` packages to help with implementing a sparse matrix.

Then, you will implement `createFancyFeatures`, which can specify at any other features you choose to help improve performance on the classification task.

The two code blocks at the end train and evaluate two models—logistic regression with L1 and L2 regularization—using your featurization functions. Besides held-out classification accuracy with 10-fold cross-validation, you will also see the features in each class given high weights by the model.

A helpful resource for getting up to speed with vector representations of documents is the first two chapters of Delip Rao and Brian McMahan, _Natural Language Processing with PyTorch_, O'Reilly, 2019.  You should be able to <a href="https://learning.oreilly.com/library/view/natural-language-processing/9781491978221/">read it online</a> via the Northeastern Library's subscription using a <tt>northeastern.edu</tt> email address.

In [ ]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold
import numpy as np

In [ ]:
# read in the movie review corpus
def readReviews():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/cornell_reviews.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  return corpus

In [ ]:
# corpus = readReviews()
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd


This is where you will implement two functions to featurize the data.

In [ ]:
# NB: The current contents are for testing only
# This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct class for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
  #Your code here
  data = pd.DataFrame(corpus)
  vectorizer = CountVectorizer(
      token_pattern = '\w*[a-zA-Z]\w*',
      decode_error = 'ignore',
      analyzer = 'word'
  )
  texts = vectorizer.fit_transform(data.text)
  classes = data['class']
  vocab = vectorizer.get_feature_names()
  return texts,classes,vocab

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createFancyFeatures(corpus):
  #Your code here
  data = pd.DataFrame(corpus)
  vectorizer = CountVectorizer(
      decode_error = 'ignore',
      token_pattern = '\w*[a-zA-Z]\w*',
      analyzer = 'word',
      ngram_range = (1,2),
      binary = 'True'
  )
  texts = vectorizer.fit_transform(data.text)
  vocab = vectorizer.get_feature_names()
  classes = data['class']
  return texts,classes,vocab

In [ ]:
# given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class

def evaluateModel(X,y,vocab,penalty="l1"):
  # create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=KFold(n_splits=10, shuffle=True, random_state=1))
  
  # determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  # determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  class0_weight_sorted = model.coef_[0, :].argsort()
  class1_weight_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  class0_indicators = [vocab[i] for i in class0_weight_sorted[:termsToTake]]
  class1_indicators = [vocab[i] for i in class1_weight_sorted[:termsToTake]]

  if model.classes_[0] == "pos":
    return avg_score,class0_indicators,class1_indicators
  else:
    return avg_score,class1_indicators,class0_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)

In [ ]:
corpus = readReviews()

Run the following to train and evaluate two models using basic features:

In [ ]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.827000
The most informative terms for pos are: ['flaws', 'memorable', 'terrific', 'edge', 'sherri', 'excellent', 'perfectly', 'masterpiece', 'enjoyable', 'using', 'fun', 'overall', 'gas', 'solid', 'command', 'quite', 'follows', 'fantastic', 'different', 'liar']
The most informative terms for neg are: ['waste', 'mess', 'ridiculous', 'lame', 'worst', 'headed', 'awful', 'unfortunately', 'cheap', 'write', 'boring', 'superior', 'tedious', 'bad', 'jesse', 'terrible', 'poor', 'maybe', 'looks', 'jakob']
----------L2 Norm-----------
The model's average accuracy is 0.833000
The most informative terms for pos are: ['fun', 'great', 'back', 'quite', 'well', 'excellent', 'overall', 'seen', 'american', 'perfectly', 'yet', 'memorable', 'terrific', 'job', 'pulp', 'true', 'very', 'performances', 'solid', 'different']
The most informative terms for neg are: ['bad', 'unfortunately', 'worst', 'waste', 'nothing', 'only', 'script', 'boring', 'awf

To analyze the accuracy of the model using the BasicFeatures representations of the document, I have used the CountVecotrizer for computing the matrix which contains the token counts. I have used three parameters within the CountVectorizer: 
> **token_pattern** - regex keeps a check of at least one alphabetic character in the feature \\
>**analyzer** - 'word' makes sure that feature are made of words and not characters, \\
>**decode_error** = 'ignore' provides instruction on what to do if a byte sequence is given to analyze that contains characters not of the given 
encoding \\

After running the runEvaluation the accuracy obtained by the model by running Logistic Regression with L1 Norm  is **82.70%** and with L2 Norm is **83.300%**



Run the following to train and evaluate two models using extended features:

In [ ]:
X,y,vocab = createFancyFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.830500
The most informative terms for pos are: ['memorable', 'terrific', 'wonderfully', 'pace', 'hilarious', 'flaws', 's life', 'help of', 'overall', 'equally', 'perfectly', 'even if', 'films the', 'entertaining', 'deserves', 'edge', 'visually', 'thanks to', 'others', 'today']
The most informative terms for neg are: ['awful', 'mess', 'worst', 'boring', 'lame', 'ridiculous', 'to work', 'flat', 'terrible', 'unfortunately', 'poor', 'headed', 'to show', 'potential', 'should have', 'or even', 'studio', 'cheap', 'ends up', 'none']
----------L2 Norm-----------
The model's average accuracy is 0.875000
The most informative terms for pos are: ['very', 'well', 'great', 'others', 'hilarious', 'fun', 'also', 'but it', 'he is', 'performances', 'memorable', 'many', 'both', 'seen', 'as the', 'one of', 'and it', 'overall', 'yet', 'terrific']
The most informative terms for neg are: ['bad', 'nothing', 'worst', 'plot', 'unfortunately', 'boring

To analyze the accuracy of the model using the createFancyFeatures representations of the document, I have used the CountVecotrizer for computing the matrix which contains the token counts. I have used five parameters within the CountVectorizer:

>**token_pattern** - regex keeps a check of at least one alphabetic character in the feature \\
>**analyzer** - 'word' makes sure that feature are made of words and not 
characters \\
>**decode_error** = 'ignore' provides instruction on what to do if a byte sequence is given to analyze that contains characters not of 
the given encoding \\
>**ngram_range** - (1,2) compute both unigram and bigram tokens \\
>**binary** = 'True', to imporve the accuracy of the model as If True, all non zero counts are set to 1. This is useful for discrete probabilistic models that model binary events rather 
than integer counts.\\


After running the runEvaluation the accuracy obtained by the model by running Logistic Regression with L1 Norm is 83.10% and with L2 Norm is 87.50% \\
Even by using the FancyFeatures model didn't significantly, the unigrams and bigrams improved the model to a small extend by adding more context to it.